## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,66.99,81,87,5.01,overcast clouds
1,1,Diamantino,BR,-14.4086,-56.4461,92.34,50,96,2.55,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,34.00,64,90,8.05,overcast clouds
3,3,Lakki Marwat,PK,32.6079,70.9114,52.12,42,0,3.29,clear sky
4,4,Ostersund,SE,63.1792,14.6357,19.40,86,100,14.97,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,66.99,81,87,5.01,overcast clouds
3,3,Lakki Marwat,PK,32.6079,70.9114,52.12,42,0,3.29,clear sky
6,6,Grand Gaube,MU,-20.0064,57.6608,79.00,84,0,3.00,clear sky
7,7,Cape Town,ZA,-33.9258,18.4232,72.00,68,20,14.97,few clouds
11,11,Hobart,AU,-42.8794,147.3294,55.40,88,90,3.44,overcast clouds
12,12,San Jose,US,37.3394,-121.8950,57.00,44,40,4.85,scattered clouds
13,13,Ancud,CL,-41.8697,-73.8203,59.00,87,75,13.80,light rain
14,14,Puerto Ayora,EC,-0.7393,-90.3518,84.20,58,0,14.97,clear sky
15,15,Antalaha,MG,-14.9003,50.2788,76.44,84,5,3.91,light rain
16,16,Bredasdorp,ZA,-34.5322,20.0403,69.80,64,4,12.66,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             354
City                   354
Country                350
Lat                    354
Lng                    354
Max Temp               354
Humidity               354
Cloudiness             354
Wind Speed             354
Current Description    354
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             350
City                   350
Country                350
Lat                    350
Lng                    350
Max Temp               350
Humidity               350
Cloudiness             350
Wind Speed             350
Current Description    350
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.99,overcast clouds,-33.6500,115.3333,
3,Lakki Marwat,PK,52.12,clear sky,32.6079,70.9114,
6,Grand Gaube,MU,79.00,clear sky,-20.0064,57.6608,
7,Cape Town,ZA,72.00,few clouds,-33.9258,18.4232,
11,Hobart,AU,55.40,overcast clouds,-42.8794,147.3294,
12,San Jose,US,57.00,scattered clouds,37.3394,-121.8950,
13,Ancud,CL,59.00,light rain,-41.8697,-73.8203,
14,Puerto Ayora,EC,84.20,clear sky,-0.7393,-90.3518,
15,Antalaha,MG,76.44,light rain,-14.9003,50.2788,
16,Bredasdorp,ZA,69.80,clear sky,-34.5322,20.0403,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.99,overcast clouds,-33.6500,115.3333,Observatory Guest House
3,Lakki Marwat,PK,52.12,clear sky,32.6079,70.9114,ستاری چائے ہوٹل
6,Grand Gaube,MU,79.00,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
7,Cape Town,ZA,72.00,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
11,Hobart,AU,55.40,overcast clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
12,San Jose,US,57.00,scattered clouds,37.3394,-121.8950,The Westin San Jose
13,Ancud,CL,59.00,light rain,-41.8697,-73.8203,Hotel Arena Gruesa
14,Puerto Ayora,EC,84.20,clear sky,-0.7393,-90.3518,Castillo Galapagos
15,Antalaha,MG,76.44,light rain,-14.9003,50.2788,Hotel Vitasoa
16,Bredasdorp,ZA,69.80,clear sky,-34.5322,20.0403,Bredasdorp Country Manor


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.99,overcast clouds,-33.6500,115.3333,Observatory Guest House
3,Lakki Marwat,PK,52.12,clear sky,32.6079,70.9114,ستاری چائے ہوٹل
6,Grand Gaube,MU,79.00,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
7,Cape Town,ZA,72.00,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
11,Hobart,AU,55.40,overcast clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania


In [11]:
# 8a. Create the output File (CSV)
output_data_file = ('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
current_description = hotel_df["Current Description"]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))